In [5]:
# Dependencies and Setup
import requests
import gmaps
import pandas as pd

# Import API key
from config import g_key

In [6]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [7]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Adrar,MR,2020-04-06 21:25:25,20.50,-10.07,82.31,13,0,11.21
1,1,Astana,KZ,2020-04-06 21:22:48,51.18,71.45,37.40,86,100,11.18
2,2,Bredasdorp,ZA,2020-04-06 21:23:24,-34.53,20.04,55.99,62,15,11.41
3,3,Ushuaia,AR,2020-04-06 21:21:18,-54.80,-68.30,57.20,50,75,16.11
4,4,Hermanus,ZA,2020-04-06 21:25:26,-34.42,19.23,55.99,71,16,5.99


In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Adrar,MR,2020-04-06 21:25:25,20.50,-10.07,82.31,13,0,11.21
6,6,Saint-Pierre,RE,2020-04-06 21:25:26,-21.34,55.48,77.00,73,20,6.93
9,9,Bonthe,SL,2020-04-06 21:25:27,7.53,-12.51,81.59,85,77,6.35
15,15,Kant,IN,2020-04-06 21:25:28,27.80,79.80,75.22,24,0,7.99
18,18,Conde,BR,2020-04-06 21:25:28,-7.26,-34.91,82.40,83,20,9.17
23,23,Axim,GH,2020-04-06 21:25:29,4.87,-2.24,82.35,81,0,5.44
26,26,Souillac,MU,2020-04-06 21:25:29,-20.52,57.52,78.01,83,40,8.05
27,27,Vila Velha,BR,2020-04-06 21:24:47,-20.33,-40.29,78.80,78,75,10.29
33,33,Francisco Sa,BR,2020-04-06 21:25:30,-16.48,-43.49,86.00,55,20,4.70
35,35,Mahajanga,MG,2020-04-06 21:15:38,-15.72,46.32,78.80,88,20,1.23


In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Adrar,MR,82.31,20.50,-10.07,
6,Saint-Pierre,RE,77.00,-21.34,55.48,
9,Bonthe,SL,81.59,7.53,-12.51,
15,Kant,IN,75.22,27.80,79.80,
18,Conde,BR,82.40,-7.26,-34.91,


In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [25]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [28]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [29]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))